# RAG system evaluation

In [17]:
import os
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

# Generate questions based in the documents

This questions will be used to evaluate the system later.

## Opening files

Unstructured is the best to open and clean pdf files. It also partition the file to better undestand its structure, but we dont use the partitions in this step.

In [3]:
from unstructured.partition.auto import partition

folder_path = "../docs"

path, dirs, files = next(os.walk(folder_path))
        
documents_all = []
for file in files:
    with open( os.path.join(path, file), 'rb') as f:
        print(f'Opening file: {file}')
        
        # open file, clean, segment
        elements = partition(file=f)
        
        doc = "\n".join( [element.text for element in elements] )
            
        documents_all.append(doc)

len( documents_all )

Opening file: Manual_de_utilização_do_PJe_sem_certificado_digital.pdf
Opening file: processo_judicial_eletronico_grafica2.pdf
Opening file: TRT04 - Guia para participação em audiências_sessões com a ferramenta Zoom.pdf
Opening file: TRT04 - PJE 2.10.1 - Manual de autuação no  painel do procurador.pdf
Opening file: guiapje-tribunal.pdf
Opening file: guiapje-usuarios.pdf
Opening file: Instalação do PJePortable v. 1.13.10 - para uso do PJe.pdf
Opening file: PJe Mídias Desktop - Envio de Perícias em MP4.pdf
Opening file: TRT04 - MANUAL PJE 2.10.1 - PAINEL DO ADVOGADO.pdf
Opening file: Zoom - CDTI OAB.pdf
Opening file: PJeMídias - Acesso à gravação de audiências ou perícias.pdf
Opening file: TRT04 - Manual para advogados e peritos (cadastramento de dados bancários para créditos em processos do TRT04).pdf
Opening file: PJeMídiasDesktop - Envio de Provas em MP4.pdf
Opening file: PROCEDIMENTOS PARA ENVIO DE CÁLCULOS DO PJECALC NO PJE PARA ADVOGADOS.pdf
Opening file: guiapje-advogados.pdf
Openi

19

In [4]:
from pydantic import BaseModel, Field

class QuestionSchema(BaseModel):
    question: str = Field(..., description="The actual question text.")

class QuestionListSchema(BaseModel):
    questionList: list[QuestionSchema] = Field(..., description="A list of Question")

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_generate_questions = (
    "I'm creating a RAG system that works with knowledge base from a company.\n"
    "You are expert in RAG systems.\n"
    "I need you to create a set of questions based in the document provided.\n"
    "This questions will be used to evaluate the system later.\n"
    "The questions should be similar to what a employee would ask in the system.\n"
    "This is very important for the future of the company and my carreer!\n"
    "Output format: You should output in JSON format considering the provided schema.\n"
    "QUANTITY OF QUESTIONS: {QUANTITY}\n"
    "DOCUMENT:\n\n {DOCUMENT}\n\n"
)

prompt_generate_questions_lc = PromptTemplate.from_template(prompt_generate_questions)

In [11]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=500,
    api_key=os.getenv("OPENAI_API_KEY")
).with_structured_output(QuestionListSchema)

In [12]:
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain_generate_questions = (
    prompt_generate_questions_lc
    | llm_openai
)

In [13]:
responses = []
for doc in documents_all:
    response = chain_generate_questions.invoke({
        "DOCUMENT": doc,
        "QUANTITY": 5
    })
    responses.append(response)

In [14]:
questions = []
for response in responses:
    for question in response.questionList:
        q = {"question": question.question}
        questions.append(q)
len(questions)

95

In [16]:
questions

[{'question': 'Como posso acessar o PJe sem um certificado digital?'},
 {'question': 'Quais são as restrições para usuários que acessam o PJe sem certificado digital?'},
 {'question': 'Quais documentos são necessários para cadastrar um novo usuário sem certificado digital?'},
 {'question': 'Como um usuário existente com certificado digital pode criar uma senha de acesso ao PJe?'},
 {'question': 'Quais procedimentos devem ser seguidos pela secretaria do tribunal para cadastrar um usuário no PJe?'},
 {'question': 'Qual é o principal objetivo do sistema Processo Judicial Eletrônico (PJe)?'},
 {'question': 'Como o PJe pode reduzir o tempo necessário para chegar a uma decisão judicial?'},
 {'question': 'Quais são as principais mudanças na gestão dos tribunais com a implementação do PJe?'},
 {'question': 'De que maneira o PJe melhora a segurança e a liberdade dos usuários no sistema?'},
 {'question': 'Quais são os requisitos de infraestrutura tecnológica necessários para a instalação do PJe?

In [18]:
df_questions = pd.DataFrame(questions)
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  95 non-null     object
dtypes: object(1)
memory usage: 892.0+ bytes


In [19]:
df_questions.head()

,question
0,Como posso acessar o PJe sem um certificado di...
1,Quais são as restrições para usuários que aces...
2,Quais documentos são necessários para cadastra...
3,Como um usuário existente com certificado digi...
4,Quais procedimentos devem ser seguidos pela se...


In [21]:
df_questions.to_csv("df_questions.csv", index=False)

# File ingestion and chunking


- load pdf with pymuPDFLoader
    - partition 
- load with unstructured
    - use partion from unstrutured
- embbed
    - gemini
    - gpt4all
    - allmini (multilingual)
- create a idnex for each one

## Document loading

### Simple loading

In [24]:
from langchain.document_loaders import PyMuPDFLoader

folder_path = "../docs"

path, dirs, files = next(os.walk(folder_path))
        
documents_all = []
for file in files:
    file_path = os.path.join(path, file)
    print(f'Opening file: {file_path}')
    
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    document = "\n".join( [doc.page_content for doc in documents] )
    
    documents_all.append(document)

len(documents_all)

Opening file: ../docs/Manual_de_utilização_do_PJe_sem_certificado_digital.pdf
Opening file: ../docs/processo_judicial_eletronico_grafica2.pdf
Opening file: ../docs/TRT04 - Guia para participação em audiências_sessões com a ferramenta Zoom.pdf
Opening file: ../docs/TRT04 - PJE 2.10.1 - Manual de autuação no  painel do procurador.pdf
Opening file: ../docs/guiapje-tribunal.pdf
Opening file: ../docs/guiapje-usuarios.pdf
Opening file: ../docs/Instalação do PJePortable v. 1.13.10 - para uso do PJe.pdf
Opening file: ../docs/PJe Mídias Desktop - Envio de Perícias em MP4.pdf
Opening file: ../docs/TRT04 - MANUAL PJE 2.10.1 - PAINEL DO ADVOGADO.pdf


/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 22 of document ../docs/processo_judicial_eletronico_grafica2.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 3 of document ../docs/Zoom - CDTI OAB.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 4 of document ../docs/Zoom - CDTI OAB.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 5 of document ../docs/Zoom - CDTI OAB.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langcha

Opening file: ../docs/Zoom - CDTI OAB.pdf
Opening file: ../docs/PJeMídias - Acesso à gravação de audiências ou perícias.pdf
Opening file: ../docs/TRT04 - Manual para advogados e peritos (cadastramento de dados bancários para créditos em processos do TRT04).pdf
Opening file: ../docs/PJeMídiasDesktop - Envio de Provas em MP4.pdf
Opening file: ../docs/PROCEDIMENTOS PARA ENVIO DE CÁLCULOS DO PJECALC NO PJE PARA ADVOGADOS.pdf
Opening file: ../docs/guiapje-advogados.pdf
Opening file: ../docs/Cadastramento inicial no PJe.pdf
Opening file: ../docs/MANUAL FUNCIONAMENTO DAS PROCURADORIAS NO PJE.pdf
Opening file: ../docs/Manual.pdf
Opening file: ../docs/PJE - Processo Judicial Eletrônico.pdf


19

### Simple chunking

Just for clarification, below is given examples of chunk_size.

In [28]:
text = """
Guia
do
Rápido
  Pendentes de ciência ou de seu registro: conjunto de atos de comunicação em relação ao qual ainda 
não há registro de ciência pelo destinatário, independentemente do meio de intimação utilizado. Quando 
se tratar de citação ou notificação eletrônica na forma da Lei n.º 11.419/2006, é incluída a informação do 
prazo máximo para ciência pelo destinatário, após o que ocorrerá a ciência ficta. O ícone “lupa vermelha” 
permite que o destinatário tome ciência do ato. O ícone de resposta levará à ciência e a uma página em que 
a resposta pode ser elaborada.
  Ciência dada pelo destinatário direto ou indireto e dentro do prazo: conjunto de atos de comu­
nicação em relação ao qual houve ciência pelo destinatário ou por quem o represente, independentemente 
do meio de intimação utilizado, e cujo prazo ainda está em curso. Caso o ato não tenha prazo para resposta, 
será exibido no agrupador “Sem prazo”. O prazo limite provável é exibido no canto inferior direito, assim como 
"""

print(f"Characters count: {len(text)}")

Characters count: 997


In [29]:
text = """

Guia
do
Rápido
  Pendentes de ciência ou de seu registro: conjunto de atos de comunicação em relação ao qual ainda 
não há registro de ciência pelo destinatário, independentemente do meio de intimação utilizado. Quando 
se tratar de citação ou notificação eletrônica na forma da Lei n.º 11.419/2006, é incluída a informação do 
prazo máximo para ciência pelo destinatário, após o que ocorrerá a ciência ficta. O ícone “lupa vermelha” 
permite que o destinatário tome ciência do ato. O ícone de resposta levará à ciência e a uma página em que 
a resposta pode ser elaborada.
  Ciência dada pelo destinatário direto ou indireto e dentro do prazo: conjunto de atos de comu­
nicação em relação ao qual houve ciência pelo destinatário ou por quem o represente, independentemente 
do meio de intimação utilizado, e cujo prazo ainda está em curso. Caso o ato não tenha prazo para resposta, 
será exibido no agrupador “Sem prazo”. O prazo limite provável é exibido no canto inferior direito, assim como 
a informação do responsável pela ciência.
  Ciência dada pelo PJe e dentro do prazo:  conjunto de atos de comunicação em relação aos quais 
houve ciência ficta por decurso do prazo de graça da Lei n.º 11.419/2006, nos casos de intimação eletrônica, 
e cujo prazo ainda está em curso.
  Cujo prazo findou nos últimos 10 dias: conjunto de atos de comunicação cujo prazo expirou sem 
resposta nos últimos 10 dias.
  Sem prazo:  conjunto de atos de comunicação em relação ao qual houve ciência pelo destinatário ou 
por quem o represente ou ciência ficta, independentemente do meio de intimação utilizado, e que não têm 
prazo para resposta.
  Respondidos nos últimos 10 dias: conjunto de atos de comunicação que foram respondidos pelo 
intimado nos últimos 10 dias.
www.cnj.jus.br
15

"""

print(f"Characters count: {len(text)}")

Characters count: 1780


In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])